In [19]:
import numpy as np
import pandas as pd

In [20]:
data_path = "D:\!!!MAAykanat Dosyalar\MAA_Own_Study\organised_data_final.csv"

In [21]:
solar_energy_data = pd.read_csv(data_path, sep=";")

In [22]:
solar_energy_data.head()

,YEAR,MONTH,DAY,HOUR,WDS,MXT,MNT,TMP,PRC,PRS,RLH,SSI,GSR,OUP
0,2016,6,17,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,2016,6,17,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,2016,6,17,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,2016,6,17,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,2016,6,17,4,NaN,44754.0,44753.0,44693.0,0.0,848.3,80.0,0.0,0.0,0.0


In [24]:
# As it can see below, data has numeric values. It is better to approach regression problem to predict OUP

In [23]:
solar_energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10032 entries, 0 to 10031
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   YEAR    10032 non-null  int64  
 1   MONTH   10032 non-null  int64  
 2   DAY     10032 non-null  int64  
 3   HOUR    10032 non-null  int64  
 4   WDS     7964 non-null   float64
 5   MXT     8867 non-null   float64
 6   MNT     8867 non-null   float64
 7   TMP     8686 non-null   float64
 8   PRC     8416 non-null   float64
 9   PRS     9848 non-null   float64
 10  RLH     8686 non-null   float64
 11  SSI     9458 non-null   float64
 12  GSR     9443 non-null   float64
 13  OUP     9469 non-null   float64
dtypes: float64(10), int64(4)
memory usage: 1.1 MB


In [ ]:
# Data has some missing value almost in each features.

In [25]:
solar_energy_data.isnull().sum()

YEAR        0
MONTH       0
DAY         0
HOUR        0
WDS      2068
MXT      1165
MNT      1165
TMP      1346
PRC      1616
PRS       184
RLH      1346
SSI       574
GSR       589
OUP       563
dtype: int64